# A/B testing

In [8]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.serverless import ServerlessInferenceConfig

role = get_execution_role()

# Specify Model Image_uri
image_uri='763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-inference:1.9.1-transformers4.12.3-cpu-py38-ubuntu20.04'
sagemaker_session = sagemaker.Session()

In [13]:
# Hub Model configuration. <https://huggingface.co/models>
model1_env = {
    'HF_MODEL_ID':'distilbert-base-uncased-distilled-squad',
    'HF_TASK':'question-answering'
}

model1_name = "Model1"

# create Hugging Face Model Class
model1 = HuggingFaceModel(
   name=model1_name,
   env=model1_env,                      # configuration for loading model from Hub
   role= role,
   transformers_version="4.17",  # transformers version used
   pytorch_version="1.10",        # pytorch version used
   py_version='py38',            # python version used
   image_uri=image_uri,          # image uri
)

In [15]:
instance_type = "ml.c5.4xlarge"
container1_def = model1.prepare_container_def(instance_type)

In [16]:
sagemaker_session.create_model(
    name=model1_name, role=role, container_defs=container1_def
)


'Model1'

In [17]:
# Hub Model configuration. <https://huggingface.co/models>
model2_env = {
    'HF_MODEL_ID':'deepset/roberta-base-squad2',
    'HF_TASK':'question-answering'
}
model2_name = "Model2"

# create Hugging Face Model Class
model2 = HuggingFaceModel(
   name=model2_name,
   env=model2_env,                      # configuration for loading model from Hub
   role= role,
   transformers_version="4.17",  # transformers version used
   pytorch_version="1.10",        # pytorch version used
   py_version='py38',            # python version used
   image_uri=image_uri,          # image uri
)

In [18]:
instance_type = "ml.c5.4xlarge"
container2_def = model2.prepare_container_def(instance_type)
sagemaker_session.create_model(
    name=model2_name, role=role, container_defs=container2_def
)


'Model2'

In [19]:
from sagemaker.session import production_variant

variant1 = production_variant(
    model_name=model1_name,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant1",
    initial_weight=1,
)
variant2 = production_variant(
    model_name=model2_name,
    instance_type="ml.c5.4xlarge",
    initial_instance_count=1,
    variant_name="Variant2",
    initial_weight=1,
)

print(variant1, variant2)

{'ModelName': 'Model1', 'VariantName': 'Variant1', 'InitialVariantWeight': 1, 'InitialInstanceCount': 1, 'InstanceType': 'ml.c5.4xlarge'} {'ModelName': 'Model2', 'VariantName': 'Variant2', 'InitialVariantWeight': 1, 'InitialInstanceCount': 1, 'InstanceType': 'ml.c5.4xlarge'}


In [20]:
from datetime import datetime
endpoint_name = f"ab-testing-{datetime.now():%Y-%m-%d-%H-%M-%S}"
print(f"EndpointName={endpoint_name}")

sagemaker_session.endpoint_from_production_variants(
    name=endpoint_name, production_variants=[variant1, variant2]
)

EndpointName=ab-testing-2022-08-18-08-49-27
-----------!

'ab-testing-2022-08-18-08-49-27'

In [21]:
import json

context = r"""
The Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet's remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, with an estimated 390 billion individual trees divided into 16,000 species.
"""

question="What kind of forest is Amazon?"


data = {"context":context, "question":question}
print(data)

{'context': '\nThe Amazon rainforest (Portuguese: Floresta Amazônica or Amazônia; Spanish: Selva Amazónica, Amazonía or usually Amazonia; French: Forêt amazonienne; Dutch: Amazoneregenwoud), also known in English as Amazonia or the Amazon Jungle, is a moist broadleaf forest that covers most of the Amazon basin of South America. This basin encompasses 7,000,000 square kilometres (2,700,000 sq mi), of which 5,500,000 square kilometres (2,100,000 sq mi) are covered by the rainforest. This region includes territory belonging to nine nations. The majority of the forest is contained within Brazil, with 60% of the rainforest, followed by Peru with 13%, Colombia with 10%, and with minor amounts in Venezuela, Ecuador, Bolivia, Guyana, Suriname and French Guiana. States or departments in four nations contain "Amazonas" in their names. The Amazon represents over half of the planet\'s remaining rainforests, and comprises the largest and most biodiverse tract of tropical rainforest in the world, wi

In [43]:
sm_runtime_client = sagemaker_session.sagemaker_runtime_client
sm_client = sagemaker_session.sagemaker_client

results = {"Variant1": 0, "Variant2": 0, "total_count": 0}

# Testing 
for i in range(20):
    response = sm_runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(data))
    results[response['InvokedProductionVariant']] += 1
    results["total_count"] += 1

In [37]:
print(f"Invokations per endpoint variant: \n Variant1: {results['Variant1']/results['total_count']*100}%; \n Variant2: {results['Variant2']/results['total_count']*100}%.")

Invokations per endpoint variant: 
 Variant1: 45.0%; 
 Variant2: 55.00000000000001%.


## Update variant weights 

In [39]:
sm_client.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {"DesiredWeight": 10, "VariantName": "Variant1"},
        {"DesiredWeight": 90, "VariantName": "Variant2"},
    ],
)

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:941656036254:endpoint/ab-testing-2022-08-18-08-49-27',
 'ResponseMetadata': {'RequestId': 'b084b28d-148d-4591-a76a-dd47f691738d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b084b28d-148d-4591-a76a-dd47f691738d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '98',
   'date': 'Thu, 18 Aug 2022 13:35:17 GMT'},
  'RetryAttempts': 0}}

In [41]:
import time 

endpoint_description = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_description['EndpointStatus']
print("Status: " + status)

while status=='Updating':
    time.sleep(1)
    endpoint_description = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = endpoint_description['EndpointStatus']
    instance_count = endpoint_description['ProductionVariants'][0]['CurrentInstanceCount']
    print(f"Status: {status}")
    print(f"Current Instance count: {instance_count}")

Status: InService


In [44]:
results = {"Variant1": 0, "Variant2": 0, "total_count": 0}

# Testing 
for i in range(20):
    response = sm_runtime_client.invoke_endpoint(EndpointName=endpoint_name, ContentType="application/json", Body=json.dumps(data))
    results[response['InvokedProductionVariant']] += 1
    results["total_count"] += 1

In [45]:
print(f"Invokations per endpoint variant: \n Variant1: {results['Variant1']/results['total_count']*100}%; \n Variant2: {results['Variant2']/results['total_count']*100}%.")

Invokations per endpoint variant: 
 Variant1: 10.0%; 
 Variant2: 90.0%.


In [ ]:
sm_client.delete_endpoint(EndpointName=endpoint_name)